In [10]:
import (
    "fmt"
    "os"
    "log"
    "time"
    "strings"
)

In [10]:
type User struct {
    Email string
    Name string
}

var DataBase = []User{
    {"apple.inc@example.com", "Apple Inc"},
    {"abc.tutor@example.com", "ABC Tutor"},
    {"boss.inc@example.com", "Boss Inc"},
    {"cnn.news@example.com", "CNN News"},
    {"dell.inc@example.com", "Dell Inc"},
    {"ebay.inc@example.com", "eBay Inc"},
    {"facebook.net@example.com", "Facebook Net"},
    {"google.inc@exmple.com", "Google Inc"},
    {"hp.inc@example.com", "HP Inc"},
    {"ie.net@example.com", "IE Net"},
}

type Woker struct {
    users []User
}

func NewWorker(users []User) *Woker {
    return &Woker{users: users}
}

func (w *Woker) Find(email string) *User {
    for idx := range w.users {
        user := &w.users[idx]
        if user.Email == email {
            return user
        }
    }
    
    return nil
}

In [15]:
email := "dell.inc@example.com" //os.Args[1]
w := NewWorker(DataBase)

log.Printf("Looking for \"%s\"", email)

user := w.Find(email)
if user != nil {
    log.Printf("The email %s is owned by \"%s\"", email, user.Name)
} else {
    log.Printf("The email \"%s\" was not found.", email)
}

2018/06/09 12:20:08 Looking for "dell.inc@example.com"
2018/06/09 12:20:08 The email dell.inc@example.com is owned by "Dell Inc"


In [16]:
email := "dl.inc@example.com" //WRONG Email
w := NewWorker(DataBase)

log.Printf("Looking for \"%s\"", email)

user := w.Find(email)
if user != nil {
    log.Printf("The email %s is owned by \"%s\"", email, user.Name)
} else {
    log.Printf("The email \"%s\" was not found.", email)
}

2018/06/09 12:20:27 Looking for "dl.inc@example.com"
2018/06/09 12:20:27 The email "dl.inc@example.com" was not found.


# 改善: 使用 channel

In [2]:
type User struct {
    Email string
    Name string
}

var DataBase = []User{
    {"apple.inc@example.com", "Apple Inc"},
    {"abc.tutor@example.com", "ABC Tutor"},
    {"boss.inc@example.com", "Boss Inc"},
    {"cnn.news@example.com", "CNN News"},
    {"dell.inc@example.com", "Dell Inc"},
    {"ebay.inc@example.com", "eBay Inc"},
    {"facebook.net@example.com", "Facebook Net"},
    {"google.inc@exmple.com", "Google Inc"},
    {"hp.inc@example.com", "HP Inc"},
    {"ie.net@example.com", "IE Net"},
}

type Woker struct {
    users []User
    ch chan *User
}

func NewWorker(users []User, ch chan *User) *Woker {
    return &Woker{users: users, ch: ch}
}

func (w *Woker) Find(email string) {
    for idx := range w.users {
        user := &w.users[idx]
        if user.Email == email {
            w.ch <- user // sent to channel
        }
    }
    
}

In [20]:
email := "dell.inc@example.com"

//  使用 channel
ch := make(chan *User)
w := NewWorker(DataBase, ch)

log.Printf("Looking for \"%s\"", email)

go w.Find(email) // run in background

user := <- ch // wait and receive from channel
log.Printf("The email %s is owned by \"%s\"", email, user.Name)


2018/06/09 12:26:39 Looking for "dell.inc@example.com"
2018/06/09 12:26:39 The email dell.inc@example.com is owned by "Dell Inc"


In [ ]:
email := "dl.inc@example.com" //WRONG Email

//  使用 channel
ch := make(chan *User)
w := NewWorker(DataBase, ch)

log.Printf("Looking for \"%s\"", email)

go w.Find(email) // run in background

user := <- ch // wait and receive from channel
log.Printf("The email %s is owned by \"%s\"", email, user.Name)

2018/06/09 12:27:26 Looking for "dl.inc@example.com"
F0609 12:27:53.270974    2334 gojupyterscaffold.go:180] Failed to echo heartbeat request: interrupted system call
goroutine 5 [running]:
github.com/golang/glog.stacks(0xc42038d200, 0xc420404000, 0x72, 0x9d)
	/home/gopher/go/src/github.com/golang/glog/glog.go:769 +0xd1
github.com/golang/glog.(*loggingT).output(0x829a20, 0xc400000003, 0xc4203f6370, 0x8204b5, 0x14, 0xb4, 0x0)
	/home/gopher/go/src/github.com/golang/glog/glog.go:720 +0x358
github.com/golang/glog.(*loggingT).printf(0x829a20, 0xc400000003, 0x5653e8, 0x24, 0xc42035dfa8, 0x1, 0x1)
	/home/gopher/go/src/github.com/golang/glog/glog.go:655 +0x14e
github.com/golang/glog.Fatalf(0x5653e8, 0x24, 0xc42035dfa8, 0x1, 0x1)
	/home/gopher/go/src/github.com/golang/glog/glog.go:1148 +0x69
github.com/yunabe/lgo/jupyter/gojupyterscaffold.(*Server).Loop.func1(0xc4203ae0c0)
	/home/gopher/go/src/github.com/yunabe/lgo/jupyter/gojupyterscaffold/gojupyterscaffold.go:180 +0xf3
created by github.com/y

In [11]:
// 修正上述 DeadLock 狀況

// email := "dell.inc@example.com"
email := "dl.inc@example.com" //WRONG Email

//  使用 channel
ch := make(chan *User)
w := NewWorker(DataBase, ch)

log.Printf("Looking for \"%s\"", email)

go w.Find(email) // run in background

// 利用 select, timeout
select {
case user := <- ch: // wait and receive from channel    
    log.Printf("The email %s is owned by \"%s\"", email, user.Name)
case <- time.After(3 * time.Second):
    log.Printf("The email \"%s\" was not found.", email)
}

2018/06/09 12:35:55 Looking for "dl.inc@example.com"
2018/06/09 12:35:58 The email "dl.inc@example.com" was not found.


# 進一步改善

使用多個 workers

In [5]:
// email := "dell.inc@example.com"
email := "dl.inc@example.com" //WRONG Email

ch := make(chan *User)
go NewWorker(DataBase[:5], ch).Find(email)
go NewWorker(DataBase[5:], ch).Find(email)

log.Printf("Looking for \"%s\"", email)

// 利用 select, timeout
select {
case user := <- ch: // wait and receive from channel    
    log.Printf("The email %s is owned by \"%s\"", email, user.Name)
case <- time.After(1 * time.Second):
    log.Printf("The email \"%s\" was not found.", email)
}

2018/06/09 12:54:52 Looking for "dl.inc@example.com"
2018/06/09 12:54:53 The email "dl.inc@example.com" was not found.


In [6]:
type User struct {
    Email string
    Name string
}

var DataBase = []User{
    {"apple.inc@example.com", "Apple Inc"},
    {"abc.tutor@example.com", "ABC Tutor"},
    {"boss.inc@example.com", "Boss Inc"},
    {"cnn.news@example.com", "CNN News"},
    {"dell.inc@example.com", "Dell Inc"},
    {"ebay.inc@example.com", "eBay Inc"},
    {"facebook.net@example.com", "Facebook Net"},
    {"google.inc@exmple.com", "Google Inc"},
    {"hp.inc@example.com", "HP Inc"},
    {"ie.net@example.com", "IE Net"},
}

type Woker struct {
    users []User
    ch chan *User
    name string
}

func NewWorker(users []User, ch chan *User, name string) *Woker {
    return &Woker{users: users, ch: ch, name: name}
}

func (w *Woker) Find(email string) {
    for idx := range w.users {
        user := &w.users[idx]
        if user.Email == email {
            log.Println(">>", w.name)
            w.ch <- user // sent to channel
        }
    }
    
}

In [9]:
email := "dell.inc@example.com"
// email := "dl.inc@example.com" //WRONG Email

//  "#1": worker's number
ch := make(chan *User)
go NewWorker(DataBase[:3], ch, "#1").Find(email)
go NewWorker(DataBase[3:6], ch, "#2").Find(email)
go NewWorker(DataBase[6:], ch, "#3").Find(email)

log.Printf("Looking for \"%s\"", email)

// 利用 select, timeout
select {
case user := <- ch: // wait and receive from channel    
    log.Printf("The email %s is owned by \"%s\"", email, user.Name)
case <- time.After(1 * time.Second):
    log.Printf("The email \"%s\" was not found.", email)
}

2018/06/09 12:55:52 Looking for "dell.inc@example.com"
2018/06/09 12:55:52 >> #2
2018/06/09 12:55:52 The email dell.inc@example.com is owned by "Dell Inc"


# 如果輸入的 email 僅有部分資訊...

我們可能會得到多組答案。如何解決呢？

Hint: 使用 strings.Contains(...)

In [11]:
type User struct {
    Email string
    Name string
}

var DataBase = []User{
    {"apple.inc@example.com", "Apple Inc"},
    {"abc.tutor@example.com", "ABC Tutor"},
    {"boss.inc@example.com", "Boss Inc"},
    {"cnn.news@example.com", "CNN News"},
    {"dell.inc@example.com", "Dell Inc"},
    {"ebay.inc@example.com", "eBay Inc"},
    {"facebook.net@example.com", "Facebook Net"},
    {"google.inc@exmple.com", "Google Inc"},
    {"hp.inc@example.com", "HP Inc"},
    {"ie.net@example.com", "IE Net"},
}

type Woker struct {
    users []User
    ch chan *User
    name string
}

func NewWorker(users []User, ch chan *User, name string) *Woker {
    return &Woker{users: users, ch: ch, name: name}
}

func (w *Woker) Find(email string) {
    for idx := range w.users {
        user := &w.users[idx]
        if strings.Contains(user.Email, email) {
            w.ch <- user // sent to channel
        }
    }
    
}

In [16]:
email := "inc"
// email := "dl.inc@example.com" //WRONG Email

//  "#1": worker's number
ch := make(chan *User)
go NewWorker(DataBase[:3], ch, "#1").Find(email)
go NewWorker(DataBase[3:6], ch, "#2").Find(email)
go NewWorker(DataBase[6:], ch, "#3").Find(email)

log.Printf("Looking for \"%s\"", email)

// 利用 select, timeout
select {
case user := <- ch: // wait and receive from channel    
    log.Printf("The email %s is owned by \"%s\"", user.Email, user.Name)
case <- time.After(1 * time.Second):
    log.Printf("The email \"%s\" was not found.", email)
}

2018/06/09 13:02:55 Looking for "inc"
2018/06/09 13:02:55 The email google.inc@exmple.com is owned by "Google Inc"
3 goroutines are hanging


In [17]:
// 修正上述僅得到一組答案

email := "inc"

//  "#1": worker's number
ch := make(chan *User)
go NewWorker(DataBase[:3], ch, "#1").Find(email)
go NewWorker(DataBase[3:6], ch, "#2").Find(email)
go NewWorker(DataBase[6:], ch, "#3").Find(email)

log.Printf("Looking for \"%s\"", email)

for{
    select {
    case user := <- ch: // wait and receive from channel    
        log.Printf("The email %s is owned by \"%s\"", user.Email, user.Name)
    case <- time.After(1 * time.Second):
//         log.Printf("The email \"%s\" was not found.", email)    
        return
    }
}

2018/06/09 13:05:58 Looking for "inc"
2018/06/09 13:05:58 The email apple.inc@example.com is owned by "Apple Inc"
2018/06/09 13:05:58 The email dell.inc@example.com is owned by "Dell Inc"
2018/06/09 13:05:58 The email ebay.inc@example.com is owned by "eBay Inc"
2018/06/09 13:05:58 The email google.inc@exmple.com is owned by "Google Inc"
2018/06/09 13:05:58 The email hp.inc@example.com is owned by "HP Inc"
2018/06/09 13:05:58 The email boss.inc@example.com is owned by "Boss Inc"
